In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

if __name__ == '__main__':
    website = 'https://www.cars24.com/buy-used-car?f=make%3A%3D%3Amahindra&sort=bestmatch&serveWarrantyCount=true&gaId=1534543658.1720027803&listingSource=TabFilter&storeCityId=2378'
    response=requests.get(website)
    response.status_code
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all("a", attrs={'class': 'IIJDn'})
    link_list = []
    for link in links:
        link_list.append(link.get('href'))
    d = {"Car_Model": [], "Location": [], "EMI": [], "Price": [], "Transmission": [], "Fuel_Type": []}
    for link in link_list:
        new_webpage = requests.get(link)
        new_soup = BeautifulSoup(new_webpage.content, 'html.parser')
        # Use .get() to avoid errors if elements are not found and provide a default empty string
        d["Car_Model"].append(new_soup.find('h1', {'class': '_2Ximl'}).text.strip() if new_soup.find('h1', {'class': '_2Ximl'}) else '')
        d["Location"].append(new_soup.find('li', {'class': '_1Rvdw'}).text.rstrip("\n").strip() if new_soup.find('li', {'class': '_1Rvdw'}) else '')
        d["EMI"].append(new_soup.find("strong", attrs={"class": '_3i9_p _3d4o3'}).text.strip() if new_soup.find("strong", attrs={"class": '_3i9_p'}) else '')
        d["Price"].append(new_soup.find("div", attrs={"class":'d-flex align-items-center'}).find("strong").text if new_soup.find("div", attrs={"class": 'd-flex align-items-center'}).find("strong") else '')
        d["Transmission"].append(new_soup.find("ul", attrs={"class":'_2JSmz'}).find_all("li")[3].text if new_soup.find("ul", attrs={"class": '_2JSmz'}).find_all("li")[3] else '')
        d["Fuel_Type"].append(new_soup.find("ul", attrs={"class":'_2JSmz'}).find_all("li")[2].text if new_soup.find("ul", attrs={"class": '_2JSmz'}).find_all("li")[2] else '')

    # Create the DataFrame after the loop
    cars_df = pd.DataFrame(d)
    # Handle missing values and save to CSV
    cars_df.replace('', np.nan, inplace=True)  # Replace empty strings with NaN
    cars_df.dropna(subset=['Car_Model'], inplace=True)  # Drop rows with missing 'Car_Model' values
    cars_df.to_csv("cars.csv", header=True, index=False)